In [ ]:
import sys
import os
import glob
import re
import numpy as np
import keras

# Keras
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model
from keras.preprocessing import image

# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer

In [ ]:
# Define a flask app
app = Flask(__name__)

In [ ]:
# You can also use pretrained model from Keras
# Check https://keras.io/applications/

from keras.applications.resnet50 import ResNet50
model = ResNet50(weights='imagenet')
model.save('models/model_resnet.h5')

In [ ]:
# Model saved with Keras model.save()
MODEL_PATH = 'models/model_resnet.h5'

# Load your trained model
model = load_model(MODEL_PATH)
# Necessary
model.make_predict_function()         
print('Model loaded. Start serving...')

In [ ]:
def model_predict(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))

    # Preprocessing the image
    x = image.img_to_array(img)
    # x = np.true_divide(x, 255)
    x = np.expand_dims(x, axis=0)

    # Be careful how your trained model deals with the input
    # otherwise, it won't make correct prediction!
    x = preprocess_input(x, mode='caffe')

    preds = model.predict(x)
    return preds

In [ ]:
@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')

In [ ]:
@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['file']

        # Save the file to ./uploads
        # Use os.path.abspath('') for .ipynb files
        basepath = os.path.abspath('')

        # Use os.path.dirname('') for .py files
        # basepath = os.path.dirname(__file__)
        
        file_path = os.path.join(
            basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)

        # Make prediction
        preds = model_predict(file_path, model)

        # Process your result for human
        pred_class = preds.argmax(axis=-1)            # Simple argmax
        pred_class = decode_predictions(preds, top=1)   # ImageNet Decode
        result = str(pred_class[0][0][1])               # Convert to string
        return result
    return None

In [ ]:
# Use this if you want to use a default address and port
# app.run(debug=True)

# Use this if you want to set your address and port
app.run(host=os.getenv('IP', '0.0.0.0'), port=int(os.getenv('PORT', 4444)))